In [1]:
from dj_notebook import activate

plus = activate()

Output()

INFO HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json 
"HTTP/1.1 200 OK"

/home/epuerta/.local/share/hatch/env/pip-compile/kitchenai/H8pmQr2m/dev/lib/python3.11/site-packages/pydantic/_inte
rnal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)

/home/epuerta/.local/share/hatch/env/pip-compile/kitchenai/H8pmQr2m/dev/lib/python3.11/site-packages/deepeval/__ini
t__.py:51: UserWarning: You are using deepeval version 2.0.9, however version 2.1.1 is available. You should 
consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(

In [4]:
from deepeval_plugin.models import DataSet, Data, AnswerRelevance, Faithfulness, ContextualRelevancy, Hallucination, Toxicity
from django.db.models import Avg, OuterRef, Subquery
from django.db.models.functions import Coalesce

dataset = await DataSet.objects.aget(id=1)

# Get page number from request

# Get all data entries for this dataset with metrics in one query
entries = Data.objects.prefetch_related(
    'answerrelevance',
    'faithfulness',
    'contextualrelevancy',
    'hallucination',
    'toxicity'
).filter(
    dataset_id=1
).values(
    'source_id',
    'input',
    'output',
    'created_at',
).annotate(
    answer_relevance=Coalesce(
        Subquery(
            AnswerRelevance.objects.filter(data_id=OuterRef('id')).values('score')[:1]
        ), None
    ),
    faithfulness=Coalesce(
        Subquery(
            Faithfulness.objects.filter(data_id=OuterRef('id')).values('score')[:1]
        ), None
    ),
    contextual_relevancy=Coalesce(
        Subquery(
            ContextualRelevancy.objects.filter(data_id=OuterRef('id')).values('score')[:1]
        ), None
    ),
    hallucination=Coalesce(
        Subquery(
            Hallucination.objects.filter(data_id=OuterRef('id')).values('score')[:1]
        ), None
    ),
    toxicity=Coalesce(
        Subquery(
            Toxicity.objects.filter(data_id=OuterRef('id')).values('score')[:1]
        ), None
    )
).order_by('-created_at')



In [6]:
entries

<QuerySet [{'source_id': 82, 'input': 'who is the point of contact for submissions?', 'output': 'Chelsey Mazurek, at Incourage Community Foundation Inc., is the point of contact for submissions. Proposals should be submitted to her at 478 E. Grand Avenue, Wisconsin Rapids, WI 54494, or via email at tribunebuilding@incouragecf.org.', 'created_at': datetime.datetime(2025, 1, 3, 5, 39, 26, 495812, tzinfo=datetime.timezone.utc), 'answer_relevance': 1.0, 'faithfulness': 1.0, 'contextual_relevancy': 0.3333333333333333, 'hallucination': 0.0, 'toxicity': 0.0}, {'source_id': 81, 'input': 'list me the top three most urgent requirements ', 'output': 'The top three most urgent requirements are:\n\n1. **Proposal Submittal Deadline**: Proposals must be submitted and received by Incourage no later than 4:00 p.m. on Friday, February 13, 2015.\n2. **RSVP for Site Visit and Q&A Session**: Prospective Proposers must RSVP for the site visit by contacting Chelsey Mazurek at cmazurek@incouragecf.org or 715-